In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/heart-disease-uci/heart.csv")

In [ ]:
df.head()

In [ ]:
df.info()


All the values are either integer or float.


The features are:

        age
        sex ---->  (already mapped to 0/1)
        chest pain type (4 values)  ----> (0,1,2,3)
        resting blood pressure
        serum cholestoral in mg/dl
        fasting blood sugar > 120 mg/dl
        resting electrocardiographic results ----> ( 0,1,2)
        maximum heart rate achieved
        exercise induced angina
        oldpeak = ST depression induced by exercise relative to rest
        the slope of the peak exercise ST segment
        number of major vessels (0-3) colored by flourosopy
        thal: 3 = normal; 6 = fixed defect; 7 = reversable defect



Categorical attributes are : "cp","fbs", "restecg", "ca" and "thal"

Analysis of Categorical data

In [ ]:
temp = (df.groupby(['target']))['cp'].value_counts(normalize=True)\
.mul(100).reset_index(name = "percentage")
sns.barplot(x = "target", y = "percentage", hue = "cp", data = temp)\
.set_title("Chest Pain vs Heart Disease")

In [ ]:
temp = (df.groupby(['target']))['fbs'].value_counts(normalize=True)\
.mul(100).reset_index(name = "percentage")
sns.barplot(x = "target", y = "percentage", hue = "fbs", data = temp).set_title("FBS vs Target")

In [ ]:
temp = (df.groupby(['target']))['restecg'].value_counts(normalize=True)\
.mul(100).reset_index(name = "percentage")
sns.barplot(x = "target", y = "percentage", hue = "restecg", data = temp)\
.set_title("resting electrocardiographic results vs Heart Disease")

In [ ]:
temp = (df.groupby(['target']))['ca'].value_counts(normalize=True)\
.mul(100).reset_index(name = "percentage")
sns.barplot(x = "target", y = "percentage", hue = "ca", data = temp)\
.set_title("ca vs Heart Disease")

In [ ]:
temp = (df.groupby(['target']))['thal'].value_counts(normalize=True)\
.mul(100).reset_index(name = "percentage")
sns.barplot(x = "target", y = "percentage", hue = "thal", data = temp)\
.set_title("thal vs Heart Disease")

Checking of outliers using boxplot

In [ ]:
df.boxplot()
plt.xticks(rotation = 90)

There seem to be outliers so we will use correlation heatmap

In [ ]:
import seaborn as sns
sns.heatmap(df.corr())

In [ ]:
plt.boxplot(df.trestbps)

In [ ]:
Q3 = df.trestbps.quantile(.75)
Q1 = df.trestbps.quantile(.25)
IQR = Q3 - Q1
df = df[~((df.trestbps < Q1 - 1.5*IQR) | (df.trestbps > Q3 + 1.5*IQR))]

In [ ]:
Q3 = df.chol.quantile(.75)
Q1 = df.chol.quantile(.25)
IQR = Q3 - Q1
df = df[~((df.chol < Q1 - 1.5*IQR) | (df.chol > Q3 + 1.5*IQR))]

In [ ]:
df.boxplot()
plt.xticks(rotation = 90)

In [ ]:
sns.countplot(x="target", data=df,hue = 'sex').set_title("GENDER - Heart Diseases")

In [ ]:
df.sample()

In [ ]:
df[(df.target ==  1) & (df.sex == 1)].age.mean()   #average of male who got stroke

In [ ]:
df[(df.target ==  1) & (df.sex == 0)].age.mean()   #average of female who got stroke

In [ ]:
df[(df.target ==  1) & (df.sex == 0)].describe()[1:2]   #avg features responsible for stroke in female

In [ ]:
df[(df.target ==  1) & (df.sex == 1)].describe()[1:2]  #avg features responsible for stroke in male

In [ ]:
df[(df.target ==  0) & (df.sex == 0)].describe()[1:2]   #avg features responsible for not having stroke in female

In [ ]:
 df[(df.target ==  0) & (df.sex == 1)].describe()[1:2]    #avg features responsible for not having stroke in female

Categorising the age

In [ ]:
x = df.age.tolist()
after_x = []
for i in x:
    if i < 20:
        after_x.append("0")
    elif i < 30:
        after_x.append("1")
    elif i < 40:
        after_x.append("2")
    elif i < 50:
        after_x.append("3")
    elif i < 60:
        after_x.append("4")
    else:
        after_x.append("5")
df["age_category"] = after_x

In [ ]:
df.sample(5)

In [ ]:
temp = (df.groupby(['target']))['age_category'].value_counts(normalize=True)\
.mul(100).reset_index(name = "percentage")
sns.barplot(x = "target", y = "percentage", hue = "age_category", data = temp)\
.set_title("Age category vs Heart Disease")

Modelling

In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(df.iloc[:,:-2],df.iloc[:,-2],)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
dpen = []
for i in range(5,11):
    model = XGBClassifier(max_depth = i)
    model.fit(train_x,train_y)
    target = model.predict(test_x)
    dpen.append(accuracy_score(test_y, target))
print("Best accuracy: ",max(dpen))